In [ ]:
import numpy as np
import cupy as cp

cp.cuda.Stream.null.synchronize()

In [ ]:
arr = cp.ones(1000,500,500)

#makes us wait for gpu to finish before returning
cp.cuda.Stream.null.synchronize()